In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pickle
import time
import numpy as np
import matplotlib.font_manager as fm
import selenium
import getpass
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

In [ ]:
print(selenium.__version__)

In [ ]:
driver = webdriver.Firefox()

In [ ]:
driver.get('https://www.altnews.in/')

In [ ]:
print(driver.title)

In [ ]:
print(driver.current_url)

In [ ]:
driver.execute_script("window.scrollTo(0, 1080)") 

In [ ]:
from selenium.webdriver.common.by import By

a = driver.find_element(by=By.XPATH, value="(//div[@class='pbs-col col__xs-1_2'])[position()=574]")

In [ ]:
a.find_element(By.CSS_SELECTOR, 'a').get_attribute('href')

In [ ]:
SCROLL_PAUSE_TIME = 10
k = 1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    k = k + 1
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        time.sleep(SCROLL_PAUSE_TIME * 2)
    if k == 300:
        break
    last_height = new_height

In [ ]:
all_link = []

In [ ]:
all_link = []
for i in range(1,575):
    a = driver.find_element(by=By.XPATH, value="(//div[@class='pbs-col col__xs-1_2'])[position()=" + str(i) +"]")
    all_link.append(a.find_element(By.CSS_SELECTOR, 'a').get_attribute('href'))

textfile = open("all_link_altNews_2024_updated.txt", "w")
for element in all_link:
    textfile.write(element + "\n")
textfile.close()

In [ ]:
driver.quit()

In [ ]:
with open('all_link_altNews_2024_updated.txt') as f:
    lines = f.readlines()

In [ ]:
url_alt=[]
for x in lines:
    url_alt.append(x.replace("\n", ""))

In [ ]:
import requests


proxies = {
    'http': 'http://103.230.199.17:10000'
}


headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

url = "https://www.altnews.in/haldwani-violence-video-of-scuffle-in-mp-falsely-shared-with-communal-slurs/"

try:
    
    response = requests.get(url, headers=headers, proxies=proxies)
    response.raise_for_status()
    print(response.text)  

except requests.exceptions.RequestException as e:
    print(f"Error: {e}")


In [ ]:
for i in range(0, len(url_alt)):
    driver.get(url_alt[i])
    with open("U:/Twitter Research/Fake News Scraping/AltNews_New_Data/" + str(i) + ".html", "w", encoding='utf-8') as f:
        f.write(driver.page_source)
    time.sleep(20)
    

In [ ]:
len(url_alt)

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup


def extract_metadata_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')

    link_tag = soup.find('link', rel='canonical')
    link = link_tag['href'] if link_tag else "Link not found"


    title_tag = soup.find('meta', property='og:title')
    title = title_tag['content'] if title_tag else "Title not found"


    published_tag = soup.find('meta', property='article:published_time')
    published_date = published_tag['content'] if published_tag else "Published date not found"


    breadcrumb = soup.find('nav', class_='breadcrumb-nav yoast-breadcrumb')
    if breadcrumb:
        breadcrumb_links = breadcrumb.find_all('a')
        topic = breadcrumb_links[-1].get_text() if breadcrumb_links else "Topic not found"
    else:
        topic = "Topic not found"

    return link, title, published_date, topic


directory_path = 'V:\\SBERT All Embedding\\AltNews_New_Data\\All Webpages\\'


data = []


for filename in os.listdir(directory_path):
    if filename.endswith(".html") or filename.endswith(".htm"):
        file_path = os.path.join(directory_path, filename)
        
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        link, title, published_date, topic = extract_metadata_from_html(html_content)

        data.append((filename, link, title, published_date, topic))


df = pd.DataFrame(data, columns=['File', 'Link', 'Title', 'Published Date', 'Topic'])
print(df)
df.to_csv('extracted_metadata_with_topic.csv', index=False)
